<a href="https://colab.research.google.com/github/ManelSoengas/DL_Task1/blob/main/LSTM_Task1_12_8_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Carga de las librerías y paquetes necesarios

import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [3]:
# Datos de entrenamiento
from sklearn.utils import shuffle
# Cargar datos preprocesados
#train_data = pd.read_csv('/content/EXIST2021_training.tsv', delimiter=',', encoding='latin1')
#test_data = pd.read_csv('/content/EXIST2021_test_labeled.tsv', delimiter=',', encoding='latin1')

test_data = pd.read_csv('/content/EXIST2021_test_labeled.tsv', sep='\t')
train_data = pd.read_csv('/content/swapp_EXIST2021_training.tsv', sep='\t') #swapping data_set training
# Mezclar los datos de entrenamiento
shuffled_train_data = shuffle(train_data, random_state=42)

# Mezclar los datos de prueba
shuffled_test_data = shuffle(test_data, random_state=42)

In [4]:
shuffled_train_data.head(10)

,test_case,id,source,language,text,task1,task2
2582,EXIST2021,2583,twitter,en,"@CritFacts @NyaVixa You're repeating yourself,...",sexist,ideological-inequality
8482,EXIST2021,452,twitter,en,I eat sooo much I really hope I don t ’ get fat,non-sexist,non-sexist
8061,EXIST2021,1659,twitter,en,MichaelRapaport @ Isn t ' this the same woman ...,non-sexist,non-sexist
10914,EXIST2021,5440,twitter,es,Paulxg_ @ Ponte a trabajar que tengo que ser m...,sexist,stereotyping-dominance
10162,EXIST2021,3614,twitter,es,"SofiaBVilla @ , rebelde , español soda , stere...",non-sexist,non-sexist
5688,EXIST2021,5689,twitter,es,Cuantas vidas ha salvado esta golfa al frente ...,sexist,misogyny-non-sexual-violence
8513,EXIST2021,1468,twitter,en,"* , because that was the BIGGEST - cock block ...",non-sexist,non-sexist
8645,EXIST2021,2739,twitter,en,The Socialist British Commonwealth of India is...,non-sexist,non-sexist
9265,EXIST2021,2722,twitter,en,In Spank “ ” prank we see how a woman is trick...,sexist,sexual-violence
3274,EXIST2021,3275,twitter,en,tw!! sh•••so much fucking bullshit today in my...,sexist,stereotyping-dominance


In [4]:
import re

def anonimize_text(text):
    # Anonimizar nombres de usuario
    text = re.sub(r'@[\w_]+', '@user', text)

    # Anonimizar hashtags
    text = re.sub(r'#\w+', '#hashtag', text)

    # Anonimizar enlaces
    text = re.sub(r'https?://\S+', 'link', text)

    return text

shuffled_train_data['text_anonymized'] = shuffled_train_data['text'].apply(anonimize_text)
shuffled_test_data['text_anonymized'] = shuffled_test_data['text'].apply(anonimize_text)

In [5]:
# Convertir a minúsculas

shuffled_train_data.text_anonymized = shuffled_train_data.text_anonymized.str.lower()
shuffled_train_data.head(10)
shuffled_test_data.text_anonymized = shuffled_test_data.text_anonymized.str.lower()
shuffled_test_data.head(10)

,test_case,id,source,language,text,task1,task2,text_anonymized
1374,EXIST2021,8352,twitter,en,"if i ever catch anyone beating a women, i swea...",non-sexist,non-sexist,"if i ever catch anyone beating a women, i swea..."
2293,EXIST2021,9271,twitter,es,"""No sé si habrá un #MeToo del cine español. Cu...",non-sexist,non-sexist,"""no sé si habrá un #hashtag del cine español. ..."
3379,EXIST2021,10357,twitter,es,En el dia de hoy se sostuvo reunión con Claudi...,non-sexist,non-sexist,en el dia de hoy se sostuvo reunión con claudi...
1090,EXIST2021,8068,twitter,en,Man I love a good podcast. I highly recommend ...,non-sexist,non-sexist,man i love a good podcast. i highly recommend ...
1803,EXIST2021,8781,twitter,en,#BidenErasedWomen#BidenAintSmart#RealWomenMatt...,sexist,ideological-inequality,#hashtag#hashtag#hashtag not transgender!.#has...
2694,EXIST2021,9672,twitter,es,"tw/ transfobia ""no es opresión"" -hombres desle...",sexist,ideological-inequality,"tw/ transfobia ""no es opresión"" -hombres desle..."
461,EXIST2021,7439,twitter,en,@not_corona_ maybe its repressed antiblackness .,non-sexist,non-sexist,@user maybe its repressed antiblackness .
3346,EXIST2021,10324,twitter,es,@dara_ordonez ya nada es igual que cuando las ...,sexist,misogyny-non-sexual-violence,@user ya nada es igual que cuando las putas y ...
2297,EXIST2021,9275,twitter,es,"@modadulce Estoy de acuerdo, me parece que ten...",non-sexist,non-sexist,"@user estoy de acuerdo, me parece que tenemos ..."
2873,EXIST2021,9851,twitter,es,#BCSinAdicciones| FAIMUDEl día de hoy realizam...,non-sexist,non-sexist,#hashtag| faimudel día de hoy realizamos una m...


In [6]:
# Eliminar url/website que no usan http, solo utilizan .com websites
# Palabras separadas por a . no son eliminadas

shuffled_train_data.text_anonymized = shuffled_train_data.text_anonymized.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
shuffled_train_data.head(10)
shuffled_test_data.text_anonymized = shuffled_test_data.text_anonymized.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
shuffled_test_data.head(10)

,test_case,id,source,language,text,task1,task2,text_anonymized
1374,EXIST2021,8352,twitter,en,"if i ever catch anyone beating a women, i swea...",non-sexist,non-sexist,"if i ever catch anyone beating a women, i swea..."
2293,EXIST2021,9271,twitter,es,"""No sé si habrá un #MeToo del cine español. Cu...",non-sexist,non-sexist,"""no sé si habrá un #hashtag del cine español. ..."
3379,EXIST2021,10357,twitter,es,En el dia de hoy se sostuvo reunión con Claudi...,non-sexist,non-sexist,en el dia de hoy se sostuvo reunión con claudi...
1090,EXIST2021,8068,twitter,en,Man I love a good podcast. I highly recommend ...,non-sexist,non-sexist,man i love a good podcast. i highly recommend ...
1803,EXIST2021,8781,twitter,en,#BidenErasedWomen#BidenAintSmart#RealWomenMatt...,sexist,ideological-inequality,#hashtag#hashtag#hashtag not transgender!.#has...
2694,EXIST2021,9672,twitter,es,"tw/ transfobia ""no es opresión"" -hombres desle...",sexist,ideological-inequality,"tw/ transfobia ""no es opresión"" -hombres desle..."
461,EXIST2021,7439,twitter,en,@not_corona_ maybe its repressed antiblackness .,non-sexist,non-sexist,@user maybe its repressed antiblackness .
3346,EXIST2021,10324,twitter,es,@dara_ordonez ya nada es igual que cuando las ...,sexist,misogyny-non-sexual-violence,@user ya nada es igual que cuando las putas y ...
2297,EXIST2021,9275,twitter,es,"@modadulce Estoy de acuerdo, me parece que ten...",non-sexist,non-sexist,"@user estoy de acuerdo, me parece que tenemos ..."
2873,EXIST2021,9851,twitter,es,#BCSinAdicciones| FAIMUDEl día de hoy realizam...,non-sexist,non-sexist,#hashtag| faimudel día de hoy realizamos una m...


In [7]:
# Eliminar &text; etiquetas html

shuffled_train_data.text_anonymized = shuffled_train_data.text_anonymized.apply(lambda x: re.sub(r'&[a-zñ]+;', '', x))
shuffled_train_data.iloc[4000:4010]
shuffled_test_data.text_anonymized = shuffled_test_data.text_anonymized.apply(lambda x: re.sub(r'&[a-zñ]+;', '', x))
shuffled_test_data.iloc[4000:4010]

,test_case,id,source,language,text,task1,task2,text_anonymized
4232,EXIST2021,11210,twitter,es,@DrManuelGarcia Jajaja de 100// 90 pero por de...,sexist,stereotyping-dominance,@user jajaja de 100// 90 pero por desgracia si...
116,EXIST2021,7094,twitter,en,@s00m3u get your fancam out of the tomasito ha...,sexist,misogyny-non-sexual-violence,@user get your fancam out of the tomasito hash...
3545,EXIST2021,10523,twitter,es,@Luis_Delia Cuánta mierda y escoria kiechneris...,non-sexist,non-sexist,@user cuánta mierda y escoria kiechnerista ......
740,EXIST2021,7718,gab,en,Lying falase victim democrat agent Joy Villa w...,sexist,ideological-inequality,lying falase victim democrat agent joy villa w...
4199,EXIST2021,11177,twitter,es,@HStylesSP @Harry_Styles Empoderamiento con es...,non-sexist,non-sexist,@user @user empoderamiento con esta cancioooon...
1974,EXIST2021,8952,gab,en,.......who do you think put that meme of #mans...,sexist,misogyny-non-sexual-violence,.......who do you think put that meme of #hash...
1081,EXIST2021,8059,twitter,en,Isaias Afewerki &amp; @AbiyAhmedAli both lied ...,non-sexist,non-sexist,isaias afewerki @user both lied about eritrea...
3675,EXIST2021,10653,gab,es,Coletas mandado callar a la feminazi,sexist,ideological-inequality,coletas mandado callar a la feminazi
1810,EXIST2021,8788,twitter,en,@YourQueenBri1 @BearsgetWATSON @teddy420180 @A...,sexist,misogyny-non-sexual-violence,@user @user @user @user since when was it okay...
830,EXIST2021,7808,gab,en,"Yeah, here you come knockin' on my door, baby ...",sexist,objectification,"yeah, here you come knockin' on my door, baby ..."


In [8]:
# Eliminar caráctres especiales, #:)(/\='] usados en emojis y hashtags

shuffled_train_data.text_anonymized = shuffled_train_data.text_anonymized.str.replace("[^a-zA-ZñÑ#]", " ")
shuffled_train_data.head(10)
shuffled_test_data.text_anonymized = shuffled_test_data.text_anonymized.str.replace("[^a-zA-ZñÑ#]", " ")
shuffled_test_data.head(10)

<ipython-input-8-2f10188dda66>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  shuffled_train_data.text_anonymized = shuffled_train_data.text_anonymized.str.replace("[^a-zA-ZñÑ#]", " ")
<ipython-input-8-2f10188dda66>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  shuffled_test_data.text_anonymized = shuffled_test_data.text_anonymized.str.replace("[^a-zA-ZñÑ#]", " ")


,test_case,id,source,language,text,task1,task2,text_anonymized
1374,EXIST2021,8352,twitter,en,"if i ever catch anyone beating a women, i swea...",non-sexist,non-sexist,if i ever catch anyone beating a women i swea...
2293,EXIST2021,9271,twitter,es,"""No sé si habrá un #MeToo del cine español. Cu...",non-sexist,non-sexist,no s si habr un #hashtag del cine español ...
3379,EXIST2021,10357,twitter,es,En el dia de hoy se sostuvo reunión con Claudi...,non-sexist,non-sexist,en el dia de hoy se sostuvo reuni n con claudi...
1090,EXIST2021,8068,twitter,en,Man I love a good podcast. I highly recommend ...,non-sexist,non-sexist,man i love a good podcast i highly recommend ...
1803,EXIST2021,8781,twitter,en,#BidenErasedWomen#BidenAintSmart#RealWomenMatt...,sexist,ideological-inequality,#hashtag#hashtag#hashtag not transgender #has...
2694,EXIST2021,9672,twitter,es,"tw/ transfobia ""no es opresión"" -hombres desle...",sexist,ideological-inequality,tw transfobia no es opresi n hombres desle...
461,EXIST2021,7439,twitter,en,@not_corona_ maybe its repressed antiblackness .,non-sexist,non-sexist,user maybe its repressed antiblackness
3346,EXIST2021,10324,twitter,es,@dara_ordonez ya nada es igual que cuando las ...,sexist,misogyny-non-sexual-violence,user ya nada es igual que cuando las putas y ...
2297,EXIST2021,9275,twitter,es,"@modadulce Estoy de acuerdo, me parece que ten...",non-sexist,non-sexist,user estoy de acuerdo me parece que tenemos ...
2873,EXIST2021,9851,twitter,es,#BCSinAdicciones| FAIMUDEl día de hoy realizam...,non-sexist,non-sexist,#hashtag faimudel d a de hoy realizamos una m...


In [9]:
# Eliminar la columna 'text'
shuffled_train_data.drop('text', axis=1, inplace=True)
shuffled_test_data.drop('text', axis=1, inplace=True)
# Renombrar la columna 'text_anonimized' como 'text'
shuffled_train_data.rename(columns={'text_anonymized': 'text'}, inplace=True)
shuffled_train_data.head(10)
shuffled_test_data.rename(columns={'text_anonymized': 'text'}, inplace=True)
shuffled_test_data.head(10)

,test_case,id,source,language,task1,task2,text
1374,EXIST2021,8352,twitter,en,non-sexist,non-sexist,if i ever catch anyone beating a women i swea...
2293,EXIST2021,9271,twitter,es,non-sexist,non-sexist,no s si habr un #hashtag del cine español ...
3379,EXIST2021,10357,twitter,es,non-sexist,non-sexist,en el dia de hoy se sostuvo reuni n con claudi...
1090,EXIST2021,8068,twitter,en,non-sexist,non-sexist,man i love a good podcast i highly recommend ...
1803,EXIST2021,8781,twitter,en,sexist,ideological-inequality,#hashtag#hashtag#hashtag not transgender #has...
2694,EXIST2021,9672,twitter,es,sexist,ideological-inequality,tw transfobia no es opresi n hombres desle...
461,EXIST2021,7439,twitter,en,non-sexist,non-sexist,user maybe its repressed antiblackness
3346,EXIST2021,10324,twitter,es,sexist,misogyny-non-sexual-violence,user ya nada es igual que cuando las putas y ...
2297,EXIST2021,9275,twitter,es,non-sexist,non-sexist,user estoy de acuerdo me parece que tenemos ...
2873,EXIST2021,9851,twitter,es,non-sexist,non-sexist,#hashtag faimudel d a de hoy realizamos una m...


In [ ]:
# Obtener la longitud de cada secuencia de texto en el conjunto de entrenamiento
sequence_lengths = [len(seq) for seq in test['text']]

# Calcular la longitud máxima
max_len = max(sequence_lengths)
print(max_len)

NameError: ignored

In [ ]:
# Pre-procesamiento par el idioma Inglés

import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import pandas as pd

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Función para mapear las etiquetas POS a las etiquetas de WordNet

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Función para preprocesar el texto

def preprocess_text(text):
    # Eliminar usuarios identificados por @
    text = re.sub(r'@\S+', '', text)

    # Eliminar URL's y links
    text = re.sub(r'https?:\/\/\S+', '', text)

    text = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', text)

    # Elimina referencias link o video

    text = re.sub(r'{link}', '', text)
    text = re.sub(r"\[video\]", '', text)

    # Elimina entidades HTML
    text = re.sub(r'&[a-z]+;', '', text)

    # Eliminar caracteres especiales y convertir a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # Elimnar los caracteres que no coincidan con el patrón
    text = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', text)


    # Tokenizar el texto
    tokens = nltk.word_tokenize(text)

    # Eliminar stopwords en inglés
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lematizar los tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in filtered_tokens]

    # Unir los tokens procesados en un texto procesado
    #processed_text = ' '.join(lemmatized_tokens)

    #return processed_text
    return lemmatized_tokens

# Obtener la columna 'text' del DataFrame 'train' y 'test' donde 'language' es 'en'

texts_train = train.loc[train['language'] == 'en', 'text']
texts_test = test.loc[test['language'] == 'en', 'text']

# Aplicar el preprocesamiento a cada texto en la columna 'text'
preprocessed_texts_train = [preprocess_text(text) for text in texts_train]
preprocessed_texts_test = [preprocess_text(text) for text in texts_test]

# Crear un nuevo DataFrame con las columnas 'text' y 'task1' para train y test
en_df_train = pd.DataFrame({'text': preprocessed_texts_train, 'task1': train.loc[train['language'] == 'en', 'task1']})
en_df_test = pd.DataFrame({'text': preprocessed_texts_test, 'task1': test.loc[test['language'] == 'en', 'task1']})


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
en_df_train

,text,task2
0,"[call, antifeminazi, shut, fuck, vile, comment...",sexist
1,"[back, woman, brave, beautiful, bad, as, babe,...",non-sexist
2,"[wow, skirt, short, length, inch]",sexist
3,"[incredible, beautifulbut, laugh, much, read, ...",non-sexist
4,"[find, extremely, hard, believe, kelly, yr, ol...",non-sexist
...,...,...
3431,"[think, big, crisis, masculinity, damage, soci...",sexist
3432,"[bank, account, look, like, street, whore]",sexist
3433,"[lev, put, hand, upper, thigh, rubbed, circle,...",sexist
3434,"[believe, u, let, ur, wife, work, anywhere, bu...",sexist


In [ ]:
# Pre-procesamiento para el idioma Español.
import re
import spacy
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

# Descargar el modelo de SpaCy para el idioma español
spacy.cli.download("es_core_news_sm")

# Cargar el modelo de SpaCy para el idioma español
nlp = spacy.load("es_core_news_sm")

# Resto del código...

# Función para tokenizar y lematizar el texto utilizando SpaCy
def preprocess_text(text):
    # Eliminar usuarios identificados por @
    text = re.sub(r'@\S+', '', text)

    # Elimina URL's y links
    text = re.sub(r'https?:\/\/\S+', '', text)

    text = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', text)

    # Elimina referencias link o video

    text = re.sub(r'{link}', '', text)
    text = re.sub(r"\[video\]", '', text)

    # Elimina entidades HTML
    text = re.sub(r'&[a-z]+;', '', text)

    # Eliminar caracteres especiales y convertir a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # Elimnar los caracteres que no coincidan con el patrón
    text = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', text)

    # Tokenizar el texto utilizando SpaCy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]

    # Eliminar stopwords en español
    stop_words = set(stopwords.words('spanish'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Unir los tokens procesados en un texto procesado
    #processed_text = ' '.join(filtered_tokens)

    #return processed_text
    return filtered_tokens



# Obtener la columna 'text' del DataFrame 'train' donde 'language' es 'es'
texts_train = train[train['language'] == 'es']['text']
texts_test = test[test['language'] == 'es']['text']

# Aplicar el preprocesamiento a cada texto en la columna 'text'
preprocessed_texts_train = [preprocess_text(text) for text in texts_train]
preprocessed_texts_test = [preprocess_text(text) for text in texts_test]

# Crear un nuevo DataFrame con la columna 'pre
# Crear un nuevo DataFrame con las columnas 'text' y 'task1' para train y test
es_df_train = pd.DataFrame({'text': preprocessed_texts_train, 'task1': train.loc[train['language'] == 'es', 'task1']})
es_df_test = pd.DataFrame({'text': preprocessed_texts_test, 'task1': test.loc[test['language'] == 'es', 'task1']})

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
es_df_test.head()

,text,task1
2208,"[golpear, ms, fuerte, perra]",sexist
2209,"[ , castigar, tres, mes, salir, chica, venir,...",non-sexist
2210,"[ , jajajaj, haber, topar, onda, , hacer, man...",sexist
2211,"[resultar, hoy, ser, da, juventud, quedar, ms,...",non-sexist
2212,"[ , decir, creer, igualdad, feminismo, actual,...",sexist


In [ ]:
#Convertir las frases a una lista de palabras.

In [ ]:
# Combinación de los dataframes train y test de cada idioma
combined_df_train = pd.concat([en_df_train, es_df_train])
combined_df_train = combined_df_train.sample(frac=1).reset_index(drop=True)

combined_df_test = pd.concat([en_df_test, es_df_test])
combined_df_test = combined_df_test.sample(frac=1).reset_index(drop=True)

In [ ]:
# Obtener la longitud de cada secuencia de texto en el conjunto de entrenamiento
sequence_lengths = [len(seq) for seq in combined_df_train['text']]

# Calcular la longitud máxima
max_len = max(sequence_lengths)
print(max_len)

127


In [ ]:
mask = combined_df_test['task1'].isin(['sexist', 'non-sexist'])
combined_df_test = combined_df_test[mask]

In [10]:
#train_data = combined_df_train['text']
#test_data = combined_df_test['text']
Y_train = shuffled_train_data['task1']
Y_test = shuffled_test_data['task1']

In [11]:
# Preprocesamiento de los datos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(shuffled_train_data['text'])  # train_data es una lista de textos de entrenamiento

X_train = tokenizer.texts_to_sequences(shuffled_train_data['text'])
X_test = tokenizer.texts_to_sequences(shuffled_test_data['text'])


In [12]:

vocab_size = len(tokenizer.word_index) + 1

In [ ]:
print(X_train[:5])  # Imprimir los primeros 5 elementos de X_train
print(X_test[:5])  # Imprimir los primeros 5 elementos de X_test

[[2, 2, 15, 90, 4447, 653, 2672, 15, 362, 11, 114, 530, 1251, 15, 1832, 338, 50, 379, 13, 620, 8, 3189, 140, 15, 175, 228, 1599, 27, 26, 142, 213, 288, 3953, 3508, 8, 8278, 58, 1252, 15, 90, 61, 1038], [10, 994, 3509, 192, 10, 185, 691, 10, 63, 11, 114, 406], [9888, 307, 11, 47, 6, 257, 72, 27, 9889, 1, 1039, 461, 8, 995, 320, 76, 78, 563, 840, 274, 76, 78, 1600, 101, 8, 74, 4448, 9890, 517, 9891, 9892], [20035, 3510, 1, 1833, 3, 329, 3, 62, 44, 518], [20036, 629, 2281, 9893, 8279, 44, 966, 201, 5335]]
[[75, 10, 317, 16618, 580, 3640, 1, 26, 10, 2190, 8, 466, 10, 37, 221, 27, 1020, 101, 6, 150, 779, 15, 73, 340, 1, 168, 23, 75, 76, 1, 7486, 13, 76, 2361, 15, 8, 253, 74, 979, 190], [9, 12, 55, 2129, 30, 16, 66, 3156, 2281, 115, 9, 28, 1934, 18, 88, 324, 886, 1, 3, 60, 7, 1, 9, 796, 18342, 14, 939, 18, 30, 3667, 166, 37, 12, 2941, 17, 14, 3, 155, 8098, 123, 37, 12, 31, 2, 25], [17, 14, 1327, 4, 163, 28, 8758, 22, 38, 2883, 3526, 4742, 4, 20, 36, 38, 14, 657, 4, 16019, 4, 1, 161, 2150, 66

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 255  # longitud máxima de las secuencias
X_train_padded = pad_sequences(X_train, maxlen=max_len)
X_test_padded = pad_sequences(X_test, maxlen=max_len)


In [14]:
import numpy as np

#num_classes = len(np.unique(Y_train_encoded))
num_classes = len(np.unique(Y_train))
print(num_classes)

2


In [ ]:
print(combined_df_test['task1'].unique())

['non-sexist' 'sexist']


In [ ]:
mask = combined_df_test['task1'].isin(['sexist', 'non-sexist'])
combined_df_test = combined_df_test[mask]

In [14]:
#from sklearn.preprocessing import LabelEncoder

#label_encoder = LabelEncoder()
#Y_train_encoded = label_encoder.fit_transform(combined_df_train['task2'])
#Y_test_encoded = label_encoder.transform(combined_df_test['task2'])

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

num_classes = 2
# Crear una instancia de LabelEncoder
label_encoder = LabelEncoder()

# Codificar las etiquetas de texto en valores numéricos
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# Aplicar one-hot encoding a las etiquetas codificadas
Y_train_onehot = to_categorical(Y_train_encoded, num_classes=num_classes)
Y_test_onehot = to_categorical(Y_test_encoded, num_classes=num_classes)



In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.regularizers import l2

num_classes = 2  # Número de clases en tu conjunto de datos

# Construcción del modelo LSTM
model = Sequential()
model.add(Embedding(vocab_size, 255, input_length=max_len))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.1)))  # num_classes es el número de clases en tu conjunto de datos

In [15]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.activations import relu
from keras.optimizers import SGD
from keras.regularizers import l2
num_classes = 2

model = Sequential()
model.add(Embedding(vocab_size, 255,
                   input_length=max_len,
                   trainable=False)) # congelar pesos embedding

model.add(LSTM(32, activation='tanh')) # reducir unidades a 64, usar ReLU
model.add(Dropout(0.7))


model.add(Dense(16, activation='relu')) # capa densa pequena
model.add(Dropout(0.7))

model.add(Dense(num_classes, activation='softmax',kernel_regularizer=l2(0.2)))

opt = SGD(learning_rate=0.00001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# entrenamiento...

# descongelar embedding despues de 3 épocas
model.layers[0].trainable = True

In [16]:
# Compilación del modelo
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(X_train_padded)
print(X_test_padded)


[[    0     0     0 ...     8  8134  5134]
 [    0     0     0 ...    12   643 16619]
 [    0     0     0 ...  1419    71    35]
 ...
 [    0     0     0 ...  3621  1024  7934]
 [    0     0     0 ...  3398  1729   310]
 [    0     0     0 ...  1194    59  1851]]
[[   0    0    0 ...    0  240  735]
 [   0    0    0 ...    4 1013 7642]
 [   0    0    0 ...  556  188  456]
 ...
 [   0    0    0 ...  970 7318 2059]
 [   0    0    0 ... 1544    8 2141]
 [   0    0    0 ... 1270  431 7166]]


In [ ]:
print(Y_train_onehot)

[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [17]:

from keras.callbacks import EarlyStopping
# Aumentar tamaño de data de validación
VAL_SPLIT = 0.2

# Agregar early stopping
early_stop = EarlyStopping(patience=2, restore_best_weights=True)
# Entrenamiento del modelo
#model.fit(X_train_padded, Y_train_onehot, batch_size=16, epochs=4, validation_data=(X_test_padded, Y_test_onehot))
model.fit(X_train_padded, Y_train_onehot, batch_size= 32, epochs=15, validation_split=VAL_SPLIT,
          callbacks=[early_stop])

Epoch 1/15
300/300 [==============================] - 37s 97ms/step - loss: 1.1612 - accuracy: 0.5308 - val_loss: 0.8257 - val_accuracy: 0.5985
Epoch 2/15
300/300 [==============================] - 10s 33ms/step - loss: 0.7063 - accuracy: 0.6200 - val_loss: 0.6208 - val_accuracy: 0.7179
Epoch 3/15
300/300 [==============================] - 8s 26ms/step - loss: 0.5748 - accuracy: 0.7646 - val_loss: 0.5213 - val_accuracy: 0.7876
Epoch 4/15
300/300 [==============================] - 6s 19ms/step - loss: 0.4691 - accuracy: 0.8406 - val_loss: 0.4621 - val_accuracy: 0.8155
Epoch 5/15
300/300 [==============================] - 5s 16ms/step - loss: 0.3716 - accuracy: 0.8935 - val_loss: 0.4521 - val_accuracy: 0.8289
Epoch 6/15
300/300 [==============================] - 6s 20ms/step - loss: 0.3004 - accuracy: 0.9174 - val_loss: 0.5164 - val_accuracy: 0.7880
Epoch 7/15
300/300 [==============================] - 5s 17ms/step - loss: 0.2559 - accuracy: 0.9354 - val_loss: 0.5348 - val_accuracy: 0.83

In [18]:
Y_pred = model.predict(X_test_padded)


137/137 [==============================] - 1s 7ms/step


In [19]:
# Evaluación del modelo
loss, accuracy = model.evaluate(X_test_padded, Y_test_onehot)
print("Loss:", loss)
print("Accuracy:", accuracy)

137/137 [==============================] - 1s 7ms/step - loss: 0.7724 - accuracy: 0.6717
Loss: 0.7724470496177673
Accuracy: 0.6717032790184021


En una primera prueba el modelo entra en overfitting claramente.Se h arealizado un fuerte preprocesamiento de la columna text y por idioma.

11_8_23:

El modelo entra en overfitting a partir de la seguna epoca. El procesamiento es mínimo (el realizado en BERT y RoBERTa) y con aumento de datos por swapping.

12_8_23:

Se añaden configuraciones de hiperparámetros,dropout, early_stopping, LR(regularización), diferentes tasas de learning_rate, batch_size y epochs. El overfitting se estabiliza y parece que el modelo aprende. Todo y que el sistema es inestable y el score conseguido no es muy bueno.


#Training
```
Epoch 1/10
300/300 [==============================] - 173s 554ms/step - loss: 0.9370 - accuracy: 0.5168 - val_loss: 0.7103 - val_accuracy: 0.5100
Epoch 2/10
300/300 [==============================] - 151s 505ms/step - loss: 640516190371840.0000 - accuracy: 0.5246 - val_loss: 0.6882 - val_accuracy: 0.5747
Epoch 3/10
300/300 [==============================] - 146s 488ms/step - loss: 1680472.0000 - accuracy: 0.6305 - val_loss: 0.6563 - val_accuracy: 0.6995
Epoch 4/10
300/300 [==============================] - 145s 485ms/step - loss: 0.6894 - accuracy: 0.6995 - val_loss: 0.6114 - val_accuracy: 0.7408
Epoch 5/10
300/300 [==============================] - 146s 487ms/step - loss: 0.5522 - accuracy: 0.7843 - val_loss: 0.4896 - val_accuracy: 0.7917
Epoch 6/10
300/300 [==============================] - 148s 493ms/step - loss: 0.4281 - accuracy: 0.8589 - val_loss: 0.4555 - val_accuracy: 0.8230
Epoch 7/10
300/300 [==============================] - 143s 478ms/step - loss: 0.3298 - accuracy: 0.9039 - val_loss: 0.6050 - val_accuracy: 0.8460
Epoch 8/10
300/300 [==============================] - 156s 521ms/step - loss: 0.2513 - accuracy: 0.9326 - val_loss: 0.5433 - val_accuracy: 0.8180
<keras.callbacks.History at 0x7b20b00d8880>
```
#Evaluación


```
Loss: 0.796480119228363
Accuracy: 0.692307710647583
```
Cambiando la función de activación de LSTM se consigue que el modelo sea estable. También se ha reducido la profundidad de la arquitectura, haciéndola más sencilla.

#Training:
```
Epoch 1/15
300/300 [==============================] - 40s 102ms/step - loss: 0.9195 - accuracy: 0.5331 - val_loss: 0.7519 - val_accuracy: 0.6344
Epoch 2/15
300/300 [==============================] - 10s 32ms/step - loss: 0.6652 - accuracy: 0.6701 - val_loss: 0.5770 - val_accuracy: 0.7525
Epoch 3/15
300/300 [==============================] - 7s 24ms/step - loss: 0.5228 - accuracy: 0.8023 - val_loss: 0.4825 - val_accuracy: 0.7951
Epoch 4/15
300/300 [==============================] - 7s 22ms/step - loss: 0.3926 - accuracy: 0.8669 - val_loss: 0.4307 - val_accuracy: 0.8335
Epoch 5/15
300/300 [==============================] - 6s 19ms/step - loss: 0.3054 - accuracy: 0.9065 - val_loss: 0.4096 - val_accuracy: 0.8381
Epoch 6/15
300/300 [==============================] - 5s 16ms/step - loss: 0.2387 - accuracy: 0.9297 - val_loss: 0.4386 - val_accuracy: 0.8564
Epoch 7/15
300/300 [==============================] - 6s 20ms/step - loss: 0.1916 - accuracy: 0.9433 - val_loss: 0.4685 - val_accuracy: 0.8740
```
#Evaluación:
```
137/137 [==============================] - 1s 7ms/step - loss: 0.7822 - accuracy: 0.6891
Loss: 0.782247006893158
Accuracy: 0.6891025900840759
```

lr = 0.2 y lr = 0.00001

#training:

```
Epoch 1/15
300/300 [==============================] - 36s 93ms/step - loss: 1.0198 - accuracy: 0.5290 - val_loss: 0.7743 - val_accuracy: 0.5789
Epoch 2/15
300/300 [==============================] - 11s 38ms/step - loss: 0.6950 - accuracy: 0.6120 - val_loss: 0.6311 - val_accuracy: 0.6861
Epoch 3/15
300/300 [==============================] - 7s 22ms/step - loss: 0.5780 - accuracy: 0.7451 - val_loss: 0.5283 - val_accuracy: 0.7742
Epoch 4/15
300/300 [==============================] - 5s 18ms/step - loss: 0.4619 - accuracy: 0.8306 - val_loss: 0.4590 - val_accuracy: 0.8172
Epoch 5/15
300/300 [==============================] - 5s 18ms/step - loss: 0.3589 - accuracy: 0.8885 - val_loss: 0.4922 - val_accuracy: 0.8180
Epoch 6/15
300/300 [==============================] - 5s 18ms/step - loss: 0.2875 - accuracy: 0.9143 - val_loss: 0.4956 - val_accuracy: 0.8456
```
#Evaluación:

```
137/137 [==============================] - 1s 8ms/step - loss: 0.6687 - accuracy: 0.6928
Loss: 0.6687313914299011
Accuracy: 0.6927655935287476

```
El mejor resultado se encuentra con una arquitetura simple LSTM, dropout, early_stopping, SGD, lr = 0.00001 y regularización LR = 0.2.

Aumentando los datos y con otras arquitecturas/modelos se puede mejorar los resultados.